In [2]:
import pandas as pd

In [7]:
labels = pd.read_csv('labels_final_insta.csv')
faces = pd.read_csv('faces_final_insta.csv')
colors = pd.read_csv('dominantcolors_insta.csv')
historical = pd.read_excel('Instagram_Historical.xlsx')
contrasts = pd.read_csv('contrasts_insta.csv')

In [10]:
print(faces.shape)
print(labels.shape)
print(colors.shape)
print(historical.shape)
print(contrasts.shape)

(1557, 7)
(1557, 5)
(1557, 2)
(1578, 19)
(1557, 2)


In [20]:
historical['Photo'][0].split('/')[5].split('?')[0]

'121824333_1780170455456980_1945710384918790628_n.jpg'

In [21]:
def get_image_id(link):
    name = link.split('/')[5].split('?')[0]
    return name

In [24]:
historical = historical[historical['Link'].notna()]
historical['image_id'] = historical['Photo'].apply(get_image_id)

In [26]:
def chop_jpg(str):
    l = str.split('.')
    return l[0]

faces['image_id'] = faces['image_id'].apply(chop_jpg)
labels['image_id'] = labels['image_id'].apply(chop_jpg)
colors['image_id'] = colors['image_id'].apply(chop_jpg)
contrasts['image_id'] = contrasts['image_id'].apply(chop_jpg)
historical['image_id'] = historical['image_id'].apply(chop_jpg)

In [28]:
face_labels = faces.merge(labels, how='inner', on='image_id')

In [30]:
face_labels_colors = face_labels.merge(colors, how='inner', on='image_id')

In [31]:
face_labels_colors_hist = face_labels_colors.merge(historical, on='image_id')

In [33]:
final = face_labels_colors_hist.merge(contrasts, how='inner', on='image_id')

In [34]:
def clear_str(s):
    s = s.replace(" ", "")
    s = s.replace('[', '')
    s = s.replace(']', '')
    s = s.replace('\'', '')
    
    return s

final['Emotions'] = final['Emotions'].apply(lambda x: clear_str(x))
final['Emotions'] = final['Emotions'].apply(lambda x: x.split(','))
final['Emotions']

0           [HAPPY]
1                []
2            [CALM]
3                []
4                []
           ...     
1549             []
1550    [SURPRISED]
1551        [HAPPY]
1552             []
1553        [HAPPY]
Name: Emotions, Length: 1554, dtype: object

In [35]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

df = pd.DataFrame(mlb.fit_transform(final['Emotions']), columns=mlb.classes_, index=None)
df = df.drop('', axis=1)
df

,ANGRY,CALM,CONFUSED,DISGUSTED,FEAR,HAPPY,SAD,SURPRISED
0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1549,0,0,0,0,0,0,0,0
1550,0,0,0,0,0,0,0,1
1551,0,0,0,0,0,1,0,0
1552,0,0,0,0,0,0,0,0


In [36]:
df = pd.concat([final, df], axis=1)
df = df.drop('Emotions', axis=1)
df

,image_id,Child,Smile,Female,Male,PeopleNumber,Text,close_up,Indoors,Outdoors,...,Overperforming Score (weighted — Likes 1x Comments 1x ),Contrast,ANGRY,CALM,CONFUSED,DISGUSTED,FEAR,HAPPY,SAD,SURPRISED
0,121824333_1780170455456980_1945710384918790628_n,True,True,True,False,1,False,False,False,False,...,1.39,HIGH,0,0,0,0,0,1,0,0
1,121968037_675216890074396_7745071755117364219_n,False,False,False,False,0,True,False,False,False,...,-1.33,HIGH,0,0,0,0,0,0,0,0
2,121969359_1514124702115403_7612990989954934559_n,True,True,False,True,2,False,False,True,False,...,-1.55,HIGH,0,1,0,0,0,0,0,0
3,121972372_384988435991392_6290850303008840320_n,True,True,False,True,1,False,False,False,False,...,1.10,HIGH,0,0,0,0,0,0,0,0
4,122026324_625512811474271_6945540037480226640_n,False,False,False,False,0,True,False,False,False,...,2.53,HIGH,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1549,246150341_277626691034227_5683683811828674721_n,False,False,False,False,0,True,False,False,False,...,-1.41,HIGH,0,0,0,0,0,0,0,0
1550,246189718_110998627919217_3496746933212668181_n,False,True,True,False,1,False,False,False,False,...,-1.62,HIGH,0,0,0,0,0,0,0,1
1551,246243318_591457412200065_2273848385719141982_n,False,True,True,False,1,False,False,False,False,...,-1.88,HIGH,0,0,0,0,0,1,0,0
1552,246461955_1046939852707517_1615270582813630673_n,True,True,True,False,1,False,False,False,False,...,-1.03,HIGH,0,0,0,0,0,0,0,0


In [37]:
df.to_csv('merged_file_insta_30-01-22.csv', index=None)